In [1]:
from sklearn.tree import DecisionTreeClassifier
# from sklearn.model_selection import train_test_split

# import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

files_name = ['Accidents0514.csv', 'Casualties0514.csv', 'Vehicles0514.csv' ]
files_path = './data/'

main_data_path = "../data/correct/"

files = [f'{files_path}{name}'for name in files_name]
files

['./data/Accidents0514.csv',
 './data/Casualties0514.csv',
 './data/Vehicles0514.csv']

## First preprocessing exec

In [2]:
df_acc = pd.read_csv(files[0], index_col=0, dtype={'LSOA_of_Accident_Location': str})
df_cas = pd.read_csv(files[1], index_col=0).drop('Vehicle_Reference', axis=1)
df_veh = pd.read_csv(files[2], index_col=0).drop('Vehicle_Reference', axis=1)

df = df_acc.join(df_cas, lsuffix='_Cas', how='inner').join(df_veh, lsuffix='_Veh', how='inner')

In [5]:
df.shape

(4287593, 64)

In [6]:
df.to_csv(f'{files_path}complete.csv')
df.to_pickle(f'{files_path}complete_pickled.pkl')

In [62]:
nan_values = df.isna().sum()
nan_values = nan_values[nan_values > 0].sort_values(ascending=False)
nan_values

LSOA_of_Accident_Location    282711
Time                            264
Location_Easting_OSGR           256
Location_Northing_OSGR          256
Longitude                       256
Latitude                        256
dtype: int64

In [33]:
f'{nan_values.shape[0]}/{df.shape[1]} columns have nan values'

'6/64 columns have nan values'

In [30]:
plt.barh(nan_values.index, nan_values, log=True)
plt.show()

In [46]:
# always 256
df[df['Latitude'].isna() & df['Longitude'].isna() & df['Location_Easting_OSGR'].isna() & df['Location_Northing_OSGR'].isna()]

,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,...,Was_Vehicle_Left_Hand_Drive?,Journey_Purpose_of_Driver,Sex_of_Driver,Age_of_Driver,Age_Band_of_Driver,Engine_Capacity_(CC),Propulsion_Code,Age_of_Vehicle,Driver_IMD_Decile,Driver_Home_Area_Type
Accident_Index,,,,,,,,,,,,,,,,,,,,,
200504AL05063,NaN,NaN,NaN,NaN,4,2,2,2,09/09/2005,6,...,1,15,1,35,6,-1,-1,-1,8,3
200504AL05063,NaN,NaN,NaN,NaN,4,2,2,2,09/09/2005,6,...,1,15,2,71,10,-1,-1,-1,8,3
200504AL05063,NaN,NaN,NaN,NaN,4,2,2,2,09/09/2005,6,...,1,15,1,35,6,-1,-1,-1,8,3
200504AL05063,NaN,NaN,NaN,NaN,4,2,2,2,09/09/2005,6,...,1,15,2,71,10,-1,-1,-1,8,3
200504AL05082,NaN,NaN,NaN,NaN,4,3,2,1,08/10/2005,7,...,1,15,3,-1,-1,1998,2,2,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008950004890,NaN,NaN,NaN,NaN,95,3,2,3,06/05/2008,3,...,1,15,1,20,4,1242,1,10,-1,1
2008950004890,NaN,NaN,NaN,NaN,95,3,2,3,06/05/2008,3,...,1,15,1,37,7,1905,2,12,-1,1
2008950005704,NaN,NaN,NaN,NaN,95,3,2,1,09/05/2008,6,...,1,15,2,52,8,-1,-1,-1,-1,1


In [71]:
df['Time'] = df['Time'].map(lambda x: int(x.split(':')[0])*60 + int(x.split(':')[1]) if type(x) == str else x)
df['Time'] = df['Time'].fillna(df['Time'].mean())

In [95]:
df['Date'] = df['Date'].map(lambda x: pd.to_datetime(x, infer_datetime_format=True ).timestamp())

In [5]:
df[['Latitude', 'Longitude', 'Location_Easting_OSGR', 'Location_Northing_OSGR']].corr()

,Latitude,Longitude,Location_Easting_OSGR,Location_Northing_OSGR
Latitude,1.000000,-0.403330,-0.393208,0.999972
Longitude,-0.403330,1.000000,0.999379,-0.401404
Location_Easting_OSGR,-0.393208,0.999379,1.000000,-0.391185
Location_Northing_OSGR,0.999972,-0.401404,-0.391185,1.000000


In [3]:
df = df.drop(['Latitude', 'Longitude', 'LSOA_of_Accident_Location'], axis=1)

In [4]:
df['Location_Easting_OSGR'] = df['Location_Easting_OSGR'].fillna(df['Location_Easting_OSGR'].mean())
df['Location_Northing_OSGR'] = df['Location_Northing_OSGR'].fillna(df['Location_Northing_OSGR'].mean())

In [11]:
df['Local_Authority_(Highway)'] = pd.factorize(df['Local_Authority_(Highway)'])[0]

In [ ]:
df.to_csv(f'{files_path}preprocessed.csv')
df.to_pickle(f'{files_path}preprocessed_pickled.pkl')

## Successive exec

In [2]:
df = pd.read_pickle(f'{files_path}preprocessed_pickled.pkl')

In [4]:
x, y = df.drop('Accident_Severity', axis=1),  df['Accident_Severity']

In [3]:
df['Accident_Severity'].value_counts()

3    3607415
2     596571
1      83607
Name: Accident_Severity, dtype: int64

In [4]:
featsFormat = [
        "Continuous", "Continuous", "Categorical",
        "Continuous", "Continuous", "Continuous",
        "Continuous","Continuous", "Categorical",
        "Categorical", "Categorical", "Continuous",
        "Categorical", "Continuous", "Categorical",
        "Continuous", "Categorical", "Continuous",
        "Continuous", "Categorical", "Categorical",
        "Categorical", "Categorical", "Categorical",
        "Categorical", "Categorical", "Continuous",
        "Continuous", "Categorical", "Categorical",
        "Categorical", "Categorical", "Categorical",
        "Categorical", "Categorical", "Categorical",
        "Categorical", "Categorical", "Categorical",
        "Categorical", "Categorical", "Categorical",
        "Categorical", "Categorical", "Categorical",
        "Categorical", "Categorical", "Continuous",
        "Continuous", "Continuous", "Continuous",
        "Categorical"]

feat_name_format = list(zip(list(df.drop(columns="Accident_Severity").columns), featsFormat))

In [5]:
cat_cols_labels = [coltype[0] for coltype in list(feat_name_format) if coltype[1] == "Categorical"]

In [6]:
import math

def round_half_up(n, decimals=0):
    multiplier = 10 ** decimals
    return math.floor(n*multiplier + 0.5) / multiplier

In [7]:
from sklearn.utils import resample

for dim in [1024, 2048, 4096, 8192, 16384, 32768, 65536, 131072, 262144, 524288, 1048576, 2097152]:
    print(f"===== {dim} ===== ")

    train_size = int(round_half_up(dim * 0.8))
    test_size = int(round_half_up(dim * 0.2))

    temp_train = resample(df, replace=False, n_samples=train_size, stratify=df["Accident_Severity"], random_state=42)
    print("Train dataset done")

    df_no_train = df.drop(temp_train.index, axis=0) # dataset - samples on train

    for col in cat_cols_labels:
        # print(f"Category {col}:")
        # print(f"\ton train: {len(temp_train[col].unique())} unique values")
        # now remove from df_no_train samples with different values
        # print(f"\ton the remainig dataset({df_no_train.shape[0]} samples): {len(df_no_train[col].unique())} unique values")
        df_no_train = df_no_train.loc[df_no_train[col].isin(temp_train[col].unique())] # only get values that are in 
        # print(f"\t\tafter the drop ({df_no_train.shape[0]} samples): {len(df_no_train[col].unique())} unique values")

    temp_test = df_no_train.sample(n=test_size, random_state=42)
    print("Test dataset done")

    t = [len([el for el in list(temp_test[cat].value_counts().index) if el not in list(temp_train[cat].value_counts().index)]) for cat in cat_cols_labels]
    consistency = all([el == 0 for el in t])
    
    print(f"Category consistency train/test: {consistency}")

    if consistency:
        temp_train.to_pickle(f'{main_data_path}input_train_{dim}.pkl')
        temp_test.to_pickle(f'{main_data_path}input_test_{dim}.pkl')

        temp_train.to_csv(f'{main_data_path}input_train_{dim}.csv')
        temp_test.to_csv(f'{main_data_path}input_test_{dim}.csv')
    else:
        print("DS not saved due to inconsistency")

===== 1024 ===== 
Train dataset done
Test dataset done
Category consistency train/test: True
===== 2048 ===== 
Train dataset done
Test dataset done
Category consistency train/test: True
===== 4096 ===== 
Train dataset done
Test dataset done
Category consistency train/test: True
===== 8192 ===== 
Train dataset done
Test dataset done
Category consistency train/test: True
===== 16384 ===== 
Train dataset done
Test dataset done
Category consistency train/test: True
===== 32768 ===== 
Train dataset done
Test dataset done
Category consistency train/test: True
===== 65536 ===== 
Train dataset done
Test dataset done
Category consistency train/test: True
===== 131072 ===== 
Train dataset done
Test dataset done
Category consistency train/test: True
===== 262144 ===== 
Train dataset done
Test dataset done
Category consistency train/test: True
===== 524288 ===== 
Train dataset done
Test dataset done
Category consistency train/test: True
===== 1048576 ===== 
Train dataset done
Test dataset done
Cat

In [11]:
# check there arent same indexes
d1 = pd.read_csv(f'{main_data_path}input_train_8192.csv', index_col=0)
d2 = pd.read_csv(f'{main_data_path}input_test_8192.csv', index_col=0)

In [14]:
for cat in cat_cols_labels:
    print(cat)
    
    e1 = d1[cat].unique()
    e1.sort()

    e2 = d2[cat].unique()
    e2.sort()

    print(e1)
    print(e2)   
    

Police_Force
[ 1  3  4  5  6  7 10 11 12 13 14 16 17 20 21 22 23 30 31 32 33 34 35 36
 37 40 41 42 43 44 45 46 47 48 50 52 53 54 55 60 61 62 63 91 92 93 94 95
 96 97 98]
[ 1  3  4  5  6  7 10 11 12 13 14 16 17 20 21 22 23 30 31 32 33 34 35 36
 37 40 41 42 43 44 45 46 47 50 52 53 54 55 60 61 62 63 91 92 93 94 95 96
 97 98]
Local_Authority_(District)
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  38  40  57
  60  61  62  63  64  65  70  71  72  73  74  75  76  77  79  80  82  83
  84  85  90  91  92  93  95 100 101 102 104 106 107 109 110 112 114 120
 121 122 123 124 126 127 128 129 130 139 140 141 142 143 144 145 146 147
 148 149 150 160 161 162 163 164 165 168 169 180 181 182 184 185 186 187
 189 200 202 203 204 206 210 211 213 215 228 231 232 233 240 241 243 245
 250 251 252 253 254 255 256 257 258 270 273 274 276 277 278 279 280 281
 282 283 284 285 286 290 291 292 293 294 300 302 303 305 306 307 

In [5]:
# import numpy as np

# # power of two
# for dim in [1024, 2048, 4096, 8192, 16384, 32768, 65536, 131072, 262144, 524288, 1048576, 2097152]:
#     temp_train, temp_test = train_test_split(df.sample(n=dim, random_state=42), test_size = 0.2)
#     print(temp_train['Accident_Severity'].value_counts())

#     temp_train.to_pickle(f'{main_data_path}input_train_{dim}.pkl')
#     temp_test.to_pickle(f'{main_data_path}input_test_{dim}.pkl')

3    692
2    109
1     18
Name: Accident_Severity, dtype: int64
3    1381
2     224
1      33
Name: Accident_Severity, dtype: int64
3    2770
2     452
1      54
Name: Accident_Severity, dtype: int64
3    5545
2     885
1     123
Name: Accident_Severity, dtype: int64
3    11073
2     1783
1      251
Name: Accident_Severity, dtype: int64
3    22154
2     3563
1      497
Name: Accident_Severity, dtype: int64
3    44123
2     7281
1     1024
Name: Accident_Severity, dtype: int64
3    88387
2    14490
1     1980
Name: Accident_Severity, dtype: int64
3    176508
2     29159
1      4048
Name: Accident_Severity, dtype: int64
3    352672
2     58505
1      8253
Name: Accident_Severity, dtype: int64
3    705289
2    116995
1     16576
Name: Accident_Severity, dtype: int64
3    1411257
2     233619
1      32845
Name: Accident_Severity, dtype: int64


## Decision Tree

In [18]:
dt = DecisionTreeClassifier()
r = dt.fit(x.iloc[:3_000_000, :], y.iloc[:3_000_000])

In [19]:
r.score(x.iloc[:3_000_000 , :], y.iloc[:3_000_000])

1.0

In [20]:
r.score(x.iloc[3_000_000: , :], y.iloc[3_000_000:])

0.8604559049326922